In [2]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Minas Gerais - Consumo de Cimento (t)'
split_index = 13 #Referente aos 14 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_MG.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),...,Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t),Minas Gerais - Consumo de Cimento (t)
0,2008,0.747173,0.703460,3.117564e+08,1.831109e+07,14.350809,2.815599e+08,8.001633,0.757434,5.483335e+08,...,0.825894,20.050304,888.816223,1175.078681,18.584944,14.242417,78.155036,2.404497,12196.722756,5902.354000
1,2009,0.745086,0.708133,3.136613e+08,1.851096e+07,14.359230,2.816401e+08,8.031078,0.759702,5.302764e+08,...,0.830594,20.118491,888.717863,1187.249071,18.626049,14.472669,79.056377,2.439866,13086.093708,6032.354000
2,2010,0.743087,0.712980,3.216212e+08,1.853410e+07,14.420260,2.834821e+08,7.994906,0.762009,5.046425e+08,...,0.835096,20.152718,888.783005,1200.816603,18.504253,14.480539,79.770914,2.516811,13948.809933,6751.429000
3,2011,0.741166,0.718012,3.283955e+08,1.849638e+07,14.438708,2.844049e+08,7.984812,0.764367,4.854622e+08,...,0.839424,19.732386,888.946710,1196.199756,18.639750,14.686379,80.239851,2.517000,14478.034725,7134.988000
4,2012,0.739310,0.723238,3.341102e+08,1.839259e+07,14.419985,2.847525e+08,9.070299,0.766787,4.867661e+08,...,0.843604,19.123801,888.986931,1216.021140,18.702693,14.531110,79.216479,2.398120,14505.212690,7144.026000
5,2012,0.735778,0.730035,3.341102e+08,1.839259e+07,14.419985,2.847525e+08,9.070299,0.770031,4.867661e+08,...,0.850598,19.123801,888.986931,1216.021140,18.702693,14.531110,79.216479,2.398120,14505.212690,7144.026000
6,2013,0.736593,0.733955,3.388207e+08,1.820659e+07,14.387080,2.847505e+08,9.529468,0.771700,5.225848e+08,...,0.850481,18.647185,889.059458,1261.577485,18.602421,14.037685,77.312871,2.248054,14035.241237,7225.971000
7,2014,0.736893,0.737451,3.422472e+08,1.797983e+07,14.350903,2.842942e+08,9.983692,0.772842,5.676413e+08,...,0.849937,19.420470,889.109318,1283.285314,18.788690,13.989516,76.228776,2.141659,13306.331980,6631.735714
8,2015,0.730297,0.748040,3.447299e+08,1.774932e+07,14.320707,2.838475e+08,10.328071,0.778416,5.179392e+08,...,0.863183,19.834213,889.120461,1287.845984,18.774990,14.000459,76.241248,2.086753,12588.656354,5836.891371
9,2016,0.729778,0.749953,3.468708e+08,1.763097e+07,14.321241,2.839239e+08,10.248043,0.778817,4.524144e+08,...,0.862927,19.849624,889.101242,1284.678488,18.570380,13.855280,76.453950,2.040880,12210.552300,5091.098914


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t)
0,0.747173,0.703460,3.117564e+08,1.831109e+07,14.350809,2.815599e+08,8.001633,0.757434,5.483335e+08,2.546644e+09,...,11.760983,0.825894,20.050304,888.816223,1175.078681,18.584944,14.242417,78.155036,2.404497,12196.722756
1,0.745086,0.708133,3.136613e+08,1.851096e+07,14.359230,2.816401e+08,8.031078,0.759702,5.302764e+08,2.679945e+09,...,9.472585,0.830594,20.118491,888.717863,1187.249071,18.626049,14.472669,79.056377,2.439866,13086.093708
2,0.743087,0.712980,3.216212e+08,1.853410e+07,14.420260,2.834821e+08,7.994906,0.762009,5.046425e+08,2.891930e+09,...,9.332485,0.835096,20.152718,888.783005,1200.816603,18.504253,14.480539,79.770914,2.516811,13948.809933
3,0.741166,0.718012,3.283955e+08,1.849638e+07,14.438708,2.844049e+08,7.984812,0.764367,4.854622e+08,3.091988e+09,...,10.985689,0.839424,19.732386,888.946710,1196.199756,18.639750,14.686379,80.239851,2.517000,14478.034725
4,0.739310,0.723238,3.341102e+08,1.839259e+07,14.419985,2.847525e+08,9.070299,0.766787,4.867661e+08,3.189506e+09,...,8.159014,0.843604,19.123801,888.986931,1216.021140,18.702693,14.531110,79.216479,2.398120,14505.212690
5,0.735778,0.730035,3.341102e+08,1.839259e+07,14.419985,2.847525e+08,9.070299,0.770031,4.867661e+08,3.189506e+09,...,8.159014,0.850598,19.123801,888.986931,1216.021140,18.702693,14.531110,79.216479,2.398120,14505.212690
6,0.736593,0.733955,3.388207e+08,1.820659e+07,14.387080,2.847505e+08,9.529468,0.771700,5.225848e+08,3.093872e+09,...,7.913561,0.850481,18.647185,889.059458,1261.577485,18.602421,14.037685,77.312871,2.248054,14035.241237
7,0.736893,0.737451,3.422472e+08,1.797983e+07,14.350903,2.842942e+08,9.983692,0.772842,5.676413e+08,2.779340e+09,...,10.350442,0.849937,19.420470,889.109318,1283.285314,18.788690,13.989516,76.228776,2.141659,13306.331980
8,0.730297,0.748040,3.447299e+08,1.774932e+07,14.320707,2.838475e+08,10.328071,0.778416,5.179392e+08,2.561731e+09,...,12.454398,0.863183,19.834213,889.120461,1287.845984,18.774990,14.000459,76.241248,2.086753,12588.656354
9,0.729778,0.749953,3.468708e+08,1.763097e+07,14.321241,2.839239e+08,10.248043,0.778817,4.524144e+08,2.516859e+09,...,13.111908,0.862927,19.849624,889.101242,1284.678488,18.570380,13.855280,76.453950,2.040880,12210.552300


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     6032.354000
1     6751.429000
2     7134.988000
3     7144.026000
4     7144.026000
5     7225.971000
6     6631.735714
7     5836.891371
8     5091.098914
9     4898.010000
10    5509.999000
11    6256.450000
12    6954.530000
13    6975.675000
14    6706.893000
15            NaN
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [16]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0) 
train_input

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t)
0,1.765991,-1.698201,-1.904908,0.610525,-0.500086,-2.231580,-1.365479,-1.737700,1.261808,-1.069662,...,1.059506,-1.840110,0.632595,-1.283871,-1.484001,-0.780787,0.290382,0.328371,0.717090,-1.543320
1,1.418351,-1.421793,-1.750379,1.165572,-0.287647,-2.153091,-1.330966,-1.438635,0.862198,-0.629794,...,-0.002029,-1.462459,0.764572,-2.055260,-1.170348,-0.249153,0.608013,0.682466,0.814295,-0.485907
2,1.085493,-1.135049,-1.104667,1.229829,1.251901,-0.350509,-1.373364,-1.134382,0.294908,0.069718,...,-0.067018,-1.100793,0.830819,-1.544381,-0.820687,-1.824423,0.618869,0.963174,1.025761,0.539816
3,0.765501,-0.837388,-0.555138,1.125084,1.717259,0.552514,-1.385196,-0.823448,-0.129560,0.729872,...,0.699865,-0.753056,0.017264,-0.260535,-0.939672,-0.071949,0.902824,1.147397,1.026282,1.169035
4,0.456460,-0.528232,-0.091560,0.836852,1.244968,0.892666,-0.112866,-0.504340,-0.100704,1.051665,...,-0.611365,-0.417195,-1.160654,0.054897,-0.428838,0.742120,0.688632,0.745362,0.699565,1.201348
5,-0.131651,-0.126096,-0.091560,0.836852,1.244968,0.892666,-0.112866,-0.076449,-0.100704,1.051665,...,-0.611365,0.144698,-1.160654,0.054897,-0.428838,0.742120,0.688632,0.745362,0.699565,1.201348
6,0.004052,0.105776,0.290553,0.320306,0.414886,0.890735,0.425339,0.143689,0.691978,0.736089,...,-0.725225,0.135308,-2.083147,0.623693,0.745232,-0.554746,0.007955,-0.002475,0.287138,0.642578
7,0.053918,0.312603,0.568514,-0.309417,-0.497701,0.444189,0.957747,0.294253,1.689098,-0.301808,...,0.405188,0.091659,-0.586450,1.014713,1.304683,1.854377,-0.058493,-0.428365,-0.005267,-0.224055
8,-1.044527,0.939006,0.769912,-0.949544,-1.259427,0.007107,1.361404,1.029298,0.589170,-1.019878,...,1.381165,1.155896,0.214351,1.102101,1.422220,1.677185,-0.043398,-0.423465,-0.156166,-1.077332
9,-1.130871,1.052157,0.943578,-1.278204,-1.245963,0.081822,1.267601,1.082238,-0.860922,-1.167948,...,1.686169,1.135322,0.244178,0.951379,1.340588,-0.969153,-0.243671,-0.339904,-0.282237,-1.526877


In [17]:
reshaped_train_input = dfToInputRNN(train_input)

In [18]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     6032.354000
1     6751.429000
2     7134.988000
3     7144.026000
4     7144.026000
5     7225.971000
6     6631.735714
7     5836.891371
8     5091.098914
9     4898.010000
10    5509.999000
11    6256.450000
12    6954.530000
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [19]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t)
13,-1.081158,0.947578,0.518937,-0.848428,-1.167678,-0.464474,0.270929,0.766751,1.163167,2.664808,...,-2.313421,0.643918,-0.139605,-1.434542,-1.6914,-1.560422,-3.22543,-3.096546,-1.391402,1.350113


In [22]:
reshaped_test_input = dfToInputRNN(test_input)
reshaped_test_input

array([[[-1.08115785,  0.94757761,  0.51893706, -0.84842846,
         -1.16767829, -0.46447426,  0.27092864,  0.76675098,
          1.16316662,  2.6648078 , -0.28415403,  0.78842482,
          2.70456197, -0.13731248,  1.0620003 ,  0.14874796,
         -0.17746697,  1.94394596,  2.11975022, -2.30353113,
          1.50318835,  0.92591867,  0.21976702,  0.38034066,
          1.07012252,  2.38027465,  1.18721136,  0.68454647,
          0.96357222,  1.49963583,  1.10064062,  1.54938258,
         -1.601903  ,  1.82071202,  1.75021424,  1.09338904,
          0.63049382, -1.89207081, -2.1907463 , -2.64430562,
          0.34649211, -1.64566109,  0.29105292, -0.8938401 ,
          1.60067003,  1.08600407,  1.57190372,  1.48403903,
          1.96565138, -2.59491426,  2.94159734,  1.66925951,
          0.0371629 ,  1.62436238,  1.59912894,  1.82219669,
          1.96888178,  1.62404736,  1.59415473,  1.92575804,
          2.73371511,  1.06901998,  1.0388819 , -1.5223207 ,
         -2.31342098,  0

In [23]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

13    6975.675
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [24]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=1000, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(592, activation='tanh', return_sequences=True, input_shape=(train_input.shape[1], 
                                                                                         train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(74, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=5000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [25]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, 0, seed)

        current_loss = min(history.history['loss'])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [26]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3782473587, 3501960926, 2402224562, 2989074210, 3693149828, 2886240282, 167218957, 1418009200, 2730918469, 122070407, 3564401658, 2499483628, 2221339883, 3116582118, 3176441566, 2823968461, 1881938094, 3186576258, 1104357248, 161162147, 1328267021, 3129529597, 1248328201, 590465523, 3730809053, 388533991, 1260188955, 2014885593, 1849691119, 298309198, 518615630, 1152889884, 2378288984, 2696336767, 1304517994, 3679547232, 2582917818, 3567208849, 1155972180, 3499300336, 1830689543, 2734246023, 3875194662, 2697355406, 1113131504, 3336753691, 1066301706, 2661989373, 2644595852, 1469707664, 3438890361, 3629963316, 3831721708, 674556066, 2057290530, 84470663, 2434885991, 1607873936, 1837130575, 3270197968, 2542583927, 1099549138, 4125692487, 2628663925, 2704712395, 2861623317, 217891135, 4233104553, 1100964242, 1055745721, 3162828931, 2257974642, 3679833477, 2241059748, 2014835494, 3616018284, 2437893906, 2640318196, 1206787193, 2175814517, 2186040748, 957063511, 3238645824, 3989706840, 331

2023-09-02 11:25:01.808369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-02 11:25:01.810190: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-09-02 11:25:02.391209: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gra

loss: 110626.8671875
winner_seed: 3782473587


Step: 1 ___________________________________________


2023-09-02 11:25:49.957438: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 11:25:49.958682: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 11:25:49.960392: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 72572.375
winner_seed: 3501960926


Step: 2 ___________________________________________


...

loss: 81548.0546875


Step: 98 ___________________________________________


2023-09-02 12:39:38.198796: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 12:39:38.199939: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 12:39:38.200604: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 116842.6953125


Step: 99 ___________________________________________


2023-09-02 12:40:22.774347: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 12:40:22.779061: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 12:40:22.781589: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 109485.2421875


final_seed: 2378288984


In [27]:
trained_model, history = lstm_model(reshaped_train_input, 
                                    train_target, want_verbose=1, seed=winner_seed)

2023-09-02 12:40:57.703069: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 12:40:57.705962: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 12:40:57.707424: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/5000


2023-09-02 12:40:58.502991: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 12:40:58.508908: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 12:40:58.510402: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 4s 4s/step - loss: 40999696.0000
Epoch 2/5000
1/1 [==============================] - 0s 21ms/step - loss: 40080168.0000
Epoch 3/5000
1/1 [==============================] - 0s 18ms/step - loss: 4670359.0000
Epoch 4/5000
1/1 [==============================] - 0s 22ms/step - loss: 1569157.2500
Epoch 5/5000
1/1 [==============================] - 0s 21ms/step - loss: 550150.8750
Epoch 6/5000
1/1 [==============================] - 0s 21ms/step - loss: 546441.0625
Epoch 7/5000
1/1 [==============================] - 0s 20ms/step - loss: 410923.2188
Epoch 8/5000
1/1 [==============================] - 0s 18ms/step - loss: 652871.3750
Epoch 9/5000
1/1 [==============================] - 0s 23ms/step - loss: 520461.8438
Epoch 10/5000
1/1 [==============================] - 0s 17ms/step - loss: 686398.0625
Epoch 11/5000
1/1 [==============================] - 0s 21ms/step - loss: 433296.0625
Epoch 12/5000
1/1 [==============================] - 0s 18ms/step - loss

1/1 [==============================] - 0s 19ms/step - loss: 144435.9062
Epoch 97/5000
1/1 [==============================] - 0s 16ms/step - loss: 143094.2188
Epoch 98/5000
1/1 [==============================] - 0s 20ms/step - loss: 151090.4531
Epoch 99/5000
1/1 [==============================] - 0s 19ms/step - loss: 141560.2500
Epoch 100/5000
1/1 [==============================] - 0s 18ms/step - loss: 166075.5781
Epoch 101/5000
1/1 [==============================] - 0s 19ms/step - loss: 214579.0000
Epoch 102/5000
1/1 [==============================] - 0s 20ms/step - loss: 181476.8906
Epoch 103/5000
1/1 [==============================] - 0s 17ms/step - loss: 134002.4062
Epoch 104/5000
1/1 [==============================] - 0s 18ms/step - loss: 136023.8594
Epoch 105/5000
1/1 [==============================] - 0s 25ms/step - loss: 135224.3125
Epoch 106/5000
1/1 [==============================] - 0s 20ms/step - loss: 152207.2344
Epoch 107/5000
1/1 [==============================] - 0s 19ms

...

1/1 [==============================] - 0s 15ms/step - loss: 198685.0000
Epoch 1966/5000
1/1 [==============================] - 0s 15ms/step - loss: 178675.7656
Epoch 1967/5000
1/1 [==============================] - 0s 15ms/step - loss: 145852.1406
Epoch 1968/5000
1/1 [==============================] - 0s 14ms/step - loss: 129480.4844
Epoch 1969/5000
1/1 [==============================] - 0s 14ms/step - loss: 158326.4219
Epoch 1970/5000
1/1 [==============================] - 0s 15ms/step - loss: 161290.1719
Epoch 1971/5000
1/1 [==============================] - 0s 15ms/step - loss: 124770.7891
Epoch 1972/5000
1/1 [==============================] - 0s 16ms/step - loss: 129832.6406
Epoch 1973/5000
1/1 [==============================] - 0s 15ms/step - loss: 168056.5781
Epoch 1974/5000
1/1 [==============================] - 0s 14ms/step - loss: 154389.4844
Epoch 1975/5000
1/1 [==============================] - 0s 15ms/step - loss: 128164.3438
Epoch 1976/5000
1/1 [===========================

1/1 [==============================] - 0s 14ms/step - loss: 158861.3906
Epoch 2059/5000
1/1 [==============================] - 0s 14ms/step - loss: 98027.7344
Epoch 2060/5000
1/1 [==============================] - 0s 15ms/step - loss: 198826.3906
Epoch 2061/5000
1/1 [==============================] - 0s 15ms/step - loss: 145239.8594
Epoch 2062/5000
1/1 [==============================] - 0s 15ms/step - loss: 249067.5312
Epoch 2063/5000
1/1 [==============================] - 0s 15ms/step - loss: 262915.2812
Epoch 2064/5000
1/1 [==============================] - 0s 17ms/step - loss: 187605.6406
Epoch 2065/5000
1/1 [==============================] - 0s 17ms/step - loss: 180620.3438
Epoch 2066/5000
1/1 [==============================] - 0s 15ms/step - loss: 143504.3125
Epoch 2067/5000
1/1 [==============================] - 0s 19ms/step - loss: 163837.1875
Epoch 2068/5000
1/1 [==============================] - 0s 16ms/step - loss: 178305.3594
Epoch 2069/5000
1/1 [============================

1/1 [==============================] - 0s 14ms/step - loss: 138801.1094
Epoch 2152/5000
1/1 [==============================] - 0s 14ms/step - loss: 160931.6875
Epoch 2153/5000
1/1 [==============================] - 0s 14ms/step - loss: 140320.3281
Epoch 2154/5000
1/1 [==============================] - 0s 19ms/step - loss: 140351.6562
Epoch 2155/5000
1/1 [==============================] - 0s 18ms/step - loss: 140380.0938
Epoch 2156/5000
1/1 [==============================] - 0s 15ms/step - loss: 237996.5938
Epoch 2157/5000
1/1 [==============================] - 0s 16ms/step - loss: 190951.6562
Epoch 2158/5000
1/1 [==============================] - 0s 16ms/step - loss: 158611.9062
Epoch 2159/5000
1/1 [==============================] - 0s 17ms/step - loss: 152462.7344
Epoch 2160/5000
1/1 [==============================] - 0s 16ms/step - loss: 133058.6094
Epoch 2161/5000
1/1 [==============================] - 0s 15ms/step - loss: 130194.8828
Epoch 2162/5000
1/1 [===========================

1/1 [==============================] - 0s 14ms/step - loss: 257729.9219
Epoch 2245/5000
1/1 [==============================] - 0s 15ms/step - loss: 212915.4062
Epoch 2246/5000
1/1 [==============================] - 0s 15ms/step - loss: 214074.5781
Epoch 2247/5000
1/1 [==============================] - 0s 15ms/step - loss: 298561.8750
Epoch 2248/5000
1/1 [==============================] - 0s 16ms/step - loss: 209269.3594
Epoch 2249/5000
1/1 [==============================] - 0s 14ms/step - loss: 211164.1562
Epoch 2250/5000
1/1 [==============================] - 0s 16ms/step - loss: 247699.5000
Epoch 2251/5000
1/1 [==============================] - 0s 14ms/step - loss: 171083.8125
Epoch 2252/5000
1/1 [==============================] - 0s 15ms/step - loss: 195405.7656
Epoch 2253/5000
1/1 [==============================] - 0s 17ms/step - loss: 231548.5938
Epoch 2254/5000
1/1 [==============================] - 0s 13ms/step - loss: 213153.5781
Epoch 2255/5000
1/1 [===========================

1/1 [==============================] - 0s 15ms/step - loss: 152779.7344
Epoch 2338/5000
1/1 [==============================] - 0s 17ms/step - loss: 167200.4844
Epoch 2339/5000
1/1 [==============================] - 0s 17ms/step - loss: 155463.5000
Epoch 2340/5000
1/1 [==============================] - 0s 13ms/step - loss: 150484.3125
Epoch 2341/5000
1/1 [==============================] - 0s 17ms/step - loss: 166411.6094
Epoch 2342/5000
1/1 [==============================] - 0s 15ms/step - loss: 149872.1094
Epoch 2343/5000
1/1 [==============================] - 0s 15ms/step - loss: 147855.4844
Epoch 2344/5000
1/1 [==============================] - 0s 15ms/step - loss: 163875.8281
Epoch 2345/5000
1/1 [==============================] - 0s 15ms/step - loss: 163303.8594
Epoch 2346/5000
1/1 [==============================] - 0s 16ms/step - loss: 146041.8125
Epoch 2347/5000
1/1 [==============================] - 0s 16ms/step - loss: 147143.2656
Epoch 2348/5000
1/1 [===========================

In [28]:
prediction = trained_model.predict(reshaped_test_input)[0][0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

2023-09-02 12:41:43.262415: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 12:41:43.263495: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 12:41:43.264321: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 929ms/step
[6975.675](test_target) - [6463.867](prediction) = 511.8078125000002


In [29]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.07337036379991903

In [30]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = lstm_model(dfToInputRNN(train_input), 
                                       dfToInputRNN(train_target), want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0][0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [31]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t)
0,1.233516,-1.217183,-0.918215,-1.407905,-0.838794,-0.751794,-0.482076,-1.221219,1.148205,-1.122043,...,1.412346,-1.233476,-1.335859,1.073554,-1.201994,0.260829,-1.413615,-1.269117,-1.049915,-1.230908
1,-0.017734,-0.014986,-0.472376,0.588399,-0.566665,-0.661457,1.392429,-0.007022,0.140893,-0.184474,...,-0.643250,0.017653,0.265921,-1.334029,-0.044300,1.073320,0.671180,0.094182,-0.295566,0.012421
2,-1.215781,1.232169,1.390591,0.819506,1.405459,1.413251,-0.910354,1.228241,-1.289098,1.306518,...,-0.769096,1.215823,1.069938,0.260475,1.246294,-1.334149,0.742435,1.174935,1.345481,1.218487


train_target:


,Minas Gerais - Consumo de Cimento (t)
0,6032.354
1,6751.429
2,7134.988


test_input:


...

test_target:


,Minas Gerais - Consumo de Cimento (t)
13,6975.675


2023-09-02 12:49:57.448768: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 12:49:57.450070: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 12:49:57.451338: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 740ms/step
Error: 511.8078125000002




[7153.89111328125,
 7139.80224609375,
 7137.8798828125,
 7280.42431640625,
 6554.412109375,
 5854.2822265625,
 5520.18798828125,
 5051.0888671875,
 5185.51318359375,
 5408.31201171875,
 6463.8671875]

In [32]:
display(targets)
display(predictions)

[7144.026,
 7144.026,
 7225.971,
 6631.735714285716,
 5836.891371428574,
 5091.098914285716,
 4898.01,
 5509.999,
 6256.45,
 6954.53,
 6975.675]

[7153.89111328125,
 7139.80224609375,
 7137.8798828125,
 7280.42431640625,
 6554.412109375,
 5854.2822265625,
 5520.18798828125,
 5051.0888671875,
 5185.51318359375,
 5408.31201171875,
 6463.8671875]

In [33]:
mae = mean_absolute_error(predictions, targets)
mae

585.6021249999993

In [34]:
porcentage = mae/np.mean(targets)
porcentage

0.0924611756980885